In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from nbody.particles import Particles
from nbody.simulation import NbodySimulation

# Sun-Earth System

In this notebook, we will test our Solar System (Sun + Earth) simulation.\
For convenice, that's define the `problem_name` here for handling data IO.

In [2]:
problem_name = "test"

Prepare physical constants

In [3]:
msun   = 1.989e33   # gram
mearth = 5.97219e27 # gram
au     = 1.496e13   # cm
day    = 86400      # sec
year   = 365*day    # sec
G      = 6.67e-8   # cgs

Re-implment the particle initialze condition of the Sun+Earth system. 

In [4]:
def initialSolarSystem(particles:Particles):
    
    num_part = 2
    G = 6.67428e-8
    AU = 1.49598e13
    mass_earth = 5.97219e27
    mass_sun = 1.989e33
    d_earth = AU * (mass_sun / (mass_earth + mass_sun))
    d_sun = - AU * (mass_earth / (mass_sun + mass_earth))
    peroid = np.sqrt(4 * np.square(np.pi) * np.power(AU, 3) / (G * (mass_earth + mass_sun)))
    vel_earth = -2 * np.pi * d_earth / peroid
    vel_sun = 2 * np.pi * d_sun / peroid
    acc_earth = -G * mass_earth / np.square(d_earth)
    acc_sun =  G * mass_sun / np.square(d_sun)

    particles = Particles(N = num_part)
    
    masses = particles.get_masses()
    masses[0, 0] = mass_sun
    masses[1, 0] = mass_earth
    
    positions = particles.get_positions()
    positions[0, 0] = d_sun
    positions[1, 0] = d_earth
    
    velocities = particles.get_velocities()
    velocities[0, 1] = vel_sun
    velocities[1, 1] = vel_earth
    
    accelerations = particles.get_accelerations()
    accelerations[0, 0] = acc_sun
    accelerations[1, 0] = acc_earth
    
    particles.set_masses(masses)
    particles.set_positions(positions)
    particles.set_velocities(velocities)
    particles.set_accelerations(accelerations)
    
    return particles

Once we initialize the particles, we could run our simulation by

In [5]:
particles = Particles(N=2)
particles = initialSolarSystem(particles)
sim = NbodySimulation(particles)
sim.setup(G=G,method="RK4",io_freq=20,io_title=problem_name,io_screen=True,visualized=False)
sim.evolve(dt=0.1*day,tmax=5*year)

AttributeError: 'Particles' object has no attribute 'get_U'

# Load data and Visualization

note: `conda install -c conda-forge ffmpeg` might be necessary

In [ ]:
import glob

In [ ]:
fns = "data_"+problem_name+"/"+"data_"+problem_name+"_[0-9][0-9][0-9][0-9][0-9].txt"
fns = glob.glob(fns)
fns.sort()
# print(fns)

In [ ]:
scale = 2 * au

fig, ax =plt.subplots()
fig.set_size_inches(10.5, 10.5, forward=True)
fig.set_dpi(72)
ol,   = ax.plot([0,au],[0,0],'ro',alpha=0.3) # the initial conditions
line, = ax.plot([],[],'o')                   # plots of particles

def init():
    ax.set_xlim(-scale,scale)
    ax.set_ylim(-scale,scale)
    ax.set_aspect('equal')
    ax.set_xlabel('X [code unit]')
    ax.set_ylabel('Y [code unit]')
    return line,

def updateParticles(frame):
    fn = fns[frame]
    m,t,x,y,z,vx,vy,vz,ax,ay,az = np.loadtxt(fn)
    # print("loadtxt done",fn)
    line.set_data(x,y)
    return line,

ani = animation.FuncAnimation(fig, updateParticles, frames=len(fns),init_func=init, blit=True)
ani.save('movie_'+problem_name+'.mp4',fps=10)